**Project goal:** 

This project is to analyze participants' narrative responses to one of the open-ended survey questions that were administered in Octobor 2021. Participants were asked about how they thought of the release date and template of the Common Data Set (CDS) survey in the field of institutional research. It is being heard that institutional research analysts in different universities have been complaining about the release date and the template of the survey. 

However, no one really gathers people's opinions on the issues. One of the reasons is that institutional research analysts are always busy with many tasks and analyzing qualitative data takes time, so people are intimidated to administer an open-ended survey, which in fact can provide in-depth information. 

Thus, the goal of this project is to utilize a supervised learning method-- K Nearnest Neighbors (KNN) to build a model to help identify the most likely label of each sentence from the responses. Two calculation methods, namely TF-IDF scores and sentence embeddings were used to compare the accuracy rates. The purpose of examing the accuracy rate is to know which calculation method is better. Additionally, a discussion will be provided regarding the possible reasons that one method is better than the other and how to improve the model.





# **Install the packages and libraries that will be used in the project.**

In [1]:
# connect to the google drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# generate the file path to access the google files.
import os
# the base Google Drive Directory
root_dir = "/content/"
# choose where you want your project files to be saved
project_folder = "drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/"
os.chdir(root_dir + project_folder)

In [3]:
!pip install -U sentence-transformers
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 5.8 MB 44.2 MB/s 
     |████████████████████████████████| 1.3 MB 45.6 MB/s 
     |████████████████████████████████| 182 kB 56.6 MB/s 
     |████████████████████████████████| 7.6 MB 52.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7b50bbda6fb03ac8e125a00f9ab7f7289841f78bdd4a1f0ac5c4399b04a41e9d
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.0 MB 630 kB/s 


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re
from collections import Counter #Q6
from gensim.parsing.preprocessing import remove_stopwords #Q2
from gensim.utils import simple_preprocess #Q6
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util #Q14
import torch #Q14
from transformers import pipeline # Q3
import mmap #Q1
import string
import faiss #Q16

# Retrieve the training data:
The raw data (159 sentences) has been split by 80/20 into train data (128 sentences) and test data (31 sentences). In the train data, the sentences were categorized into 6 labels, namely timing, template, template_accessibility, new_platform, CDS_preparation_method, and others. 

In [18]:
data_timing = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/timing.xlsx")
data_timing

,id,responses,labels
0,30_1,The release date is fine for us.,timing
1,22_1,Release in August would be ideal.,timing
2,41_1,Would be great if the template could be availa...,timing
3,44_1,It would be very helpful if they released the ...,timing
4,5_1,The release date should be by September 15th.,timing
5,25_1,Knowing when to begin information gathering an...,timing
6,37_1,"The CDS release date should, ideally, be more ...",timing
7,2_1,It would be great if the CDS were released muc...,timing
8,45_2,We begin gathering data in the summer because ...,timing
9,15_1,"Regarding the release date, consider the benef...",timing


In [20]:
timing_array = data_timing['responses']
timing_array


0                      The release date is fine for us.
1                     Release in August would be ideal.
2     Would be great if the template could be availa...
3     It would be very helpful if they released the ...
4         The release date should be by September 15th.
5     Knowing when to begin information gathering an...
6     The CDS release date should, ideally, be more ...
7     It would be great if the CDS were released muc...
8     We begin gathering data in the summer because ...
9     Regarding the release date, consider the benef...
10    We don't have all the data we need to complete...
11                             Preferably in September.
12    I believe the October time-frame is a good tim...
13                                       Mid September.
14    We all have to fill out updates with rankings ...
15    Please continue to keep the timeline to releas...
16    Lately, CDS has released the template early co...
17                                Release date i

In [21]:
data_template = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/template.xlsx")
template_array = data_template['responses']
template_array

0      Create a separate version for two-year colleges.
1                   Post the revised templates earlier.
2     Guide surveys ask for additional calculations,...
3     It would be nice if the questions aligned with...
4     The only issue I have with the format is that ...
                            ...                        
58    I'm not sure why the formatting in this year's...
59    For example, C17 could have the housing be a s...
60    Now that more schools are using the CDS as a d...
61    It is difficult to navigate because it is not ...
62    the format is painful to me- it needs to be cr...
Name: responses, Length: 63, dtype: object

In [22]:
data_template_access = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/template_accessibility.xlsx")
template_access_array = data_template_access['responses']
template_access_array

0    Posting it on our website conflicts with 508 c...
1    The spreadsheet is fine, as we add multiple su...
2    Make the formats accessible to comply with Fed...
3    If the forms were redesigned, could they be ma...
4    I have concerns about the accessibility of the...
5      The document is not formatted to be accessible.
Name: responses, dtype: object

In [23]:
data_new_platform = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/new_platform.xlsx")
new_platform_array = data_new_platform['responses']
new_platform_array

0    We would like to see a web-based database for ...
1    We need a way to compile and send the CDS as a...
2    As for format, I strongly feel that there shou...
3    Thinking long term, it would be wonderful if t...
4    Additionally, having CDS in more of a database...
5    The interface could also be used for benchmark...
6    As for the format, it would make sense for the...
7    Since we are providing all of this information...
8                         This should be top priority.
9    Regarding the questionnaire format, consider m...
Name: responses, dtype: object

In [24]:
data_prep_method = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/prep_method.xlsx")
prep_method_array = data_prep_method['responses']
prep_method_array

0    (Yes, analysts can do the calculations as comp...
1    I always do this on my own copy of the Excel s...
2    We merge all the sections onto a single Excel ...
3    But when the definitions or data needs have ch...
4    While we can’t work on the actual numbers unti...
5    Instead of grouping by publisher requested dat...
6    To be able to work with the data once, includi...
Name: responses, dtype: object

In [25]:
data_others = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/others.xlsx")
others_array = data_others['responses']
others_array

0    (Better yet, see #10 below)
Name: responses, dtype: object

# Save the training data into text files.

In [26]:
file = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/timing.txt"
np.savetxt(file, timing_array.values,  fmt='%s') # If the data is string, use  fmt='%s'. If data is number, use fmt='%d'.

In [27]:
file = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/template.txt"
np.savetxt(file, template_array.values,  fmt='%s')

In [28]:
file = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/template_access.txt"
np.savetxt(file, template_access_array.values,  fmt='%s')

In [29]:
file = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/new_platform.txt"
np.savetxt(file, new_platform_array.values,  fmt='%s')

In [30]:
file = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/prep_method.txt"
np.savetxt(file, prep_method_array.values,  fmt='%s')

In [31]:
file = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/others.txt"
np.savetxt(file, others_array.values,  fmt='%s')

# Calculate the TF-IDF score of each word:
Create a function to calculate the TF-IDF score of each word for each label first.

In [32]:
### Write you code here. 
def tf_idf_corpus(corpus):
    vectorizer = TfidfVectorizer(stop_words = "english")
    tfs = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()
    n = feature_names.shape # get the shape of the feature_names array
    tfs_array = np.array(tfs.sum(0))  # tfs.sum(0) is how we get the actual numerical representations of the tfs_array
    tfs_array.shape = n # set the shape of tfs_array the same as the feature_names array for the convenience of later computation.
    word_tfs = list(zip(feature_names, tfs_array)) # convert the zip object to a list for the convenience of further computation. format: (word, its tfidf)
    return  word_tfs # data type is "list"

In [33]:
files = ['timing', 'template','template_access','new_platform','prep_method','others']
tf_idf_list = [] # a list of arrays (6 categories)
for file in files:
    file_path = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/"+file+".txt"
    f = open(file_path, 'r')
    corpus = f.readlines()
    result = dict(tf_idf_corpus(corpus)) # tf_idf_corpus(corpus) is an array of word_tfs for each category. dict() converts it to a dictioanry. Key: word. Value: tfidf score.
    tf_idf_list.append(result)

In [34]:
len(tf_idf_list)

6

# Sum the TF-IDF scores for each sentence:
Create a function to sum the TF-IDF score for each sentence. The output is a dictionary. The key is each sentence and the value is its corresponding sum of TF-IDF scores.

In [35]:
def sum_tf_ids(name): # "name" is the name in the product_names from product_name(filepath).
    name_list = []
    dictionary = {}
    for word in name.split(" "): # e.g. name = "small world chips"
        word = word.lower().translate(str.maketrans('', '', string.punctuation))
        name_list.append(word) # e.g. name_list = ["small", "world", "chips"]
  
    sum_score = 0
    for j in range(len(tf_idf_list)):
        for word in name_list:
            if tf_idf_list[j].get(word) is not None:
                score = float(tf_idf_list[j].get(word)) # for the ditionary of tf_idf_list[j], get the tfidf score of the word.
                sum_score+=score  
            else:
                score = 0
                sum_score+=score 
                
        if j == 0:
            dictionary['timing']=sum_score
        if j == 1:
            dictionary['template']=sum_score
        if j == 2:
            dictionary['template_accessibility']=sum_score
        if j == 3:
            dictionary['new_platform']=sum_score
        if j == 4:
            dictionary['CDS preparation method']=sum_score
        if j == 5:
            dictionary['others']=sum_score
        sum_score = 0
    return dictionary

# Use the sum of TF-IDF function written above to label the test data:
The test data was retrieved as a list of sentences first, and then the list of test sentences was entered into the sum of TF-IDF function to get a predicted label for each sentence. Finally, the predicted labels were merged to the original test data, so that I was able to correct the labels if necessary and calculate the accuracy rate. The label correction and accuracy rate was done manually on the spreadsheet named "q1_tfidf_label_predicted". It was provided along with this script.

In [36]:
data_test = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/q1_test.xlsx")
test_array = data_test['responses']
test_array

0     11/1 is typically the last standard freeze dat...
1     I think that the current template formats are ...
2                                   The timing is fine.
3     release date is way too late - needs to be rel...
4     Excel template has many rows not adjusted prop...
5     I know they often offer the option of having t...
6     I find the CDS extremely useful in gathering c...
7     After fall census and prior to spring ranking ...
8     This would allow us to make sure we collect ne...
9     The main feedback I have is to request that AD...
10    If we are to post the CDS on our website, it m...
11    The analysts would finish their work and be ab...
12    Most institutions are collecting and/or finali...
13    Every year IR offices are clamoring to get the...
14    Each publisher could then just solicit the inf...
15                      We never use the Word template.
16    The release date works well with our reporting...
17    I feel like communication about timing was

In [37]:
file = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/q1_test.txt"
np.savetxt(file, test_array.values,  fmt='%s')

In [38]:
file_path = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/q1_test.txt"
f = open(file_path, 'r')
content = f.read().splitlines() # f.readlines() will have new line character \n printed at the end. This will affect further analysis.
content
#type(f.readlines()): list


["11/1 is typically the last standard freeze date across all populations, so it's not unreasonable for schools to have the CDS ready for stakeholder review on that day.",
 'I think that the current template formats are past their prime.',
 'The timing is fine.',
 'release date is way too late - needs to be released in September or earlier to collect information across departments  explore a common submission format for the publishers such as a standardized XML to eliminate re-typing',
 'Excel template has many rows not adjusted properly to display all text.',
 "I know they often offer the option of having them manually key-in our data into their systems, but I don't know of other institutions that trust that process so we end up spending a lot of time typing in to the US News Best Colleges, Princeton Review and College Board BigFuture portals.",
 'I find the CDS extremely useful in gathering common data for the various college search groups.',
 'After fall census and prior to spring ra

In [39]:
file_path = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/q1_test.txt"
f = open(file_path, 'r')
#score = 0
#label_list = []
results = []
for name in f.readlines():
  #name = line.split("\t")[0] # product name
  #label = line.split("\t")[1].replace("\n","") # category label
  #label_list.append(label)
  #max_sum_tfs = max(sum_tf_ids(name).values()) # gives the largest TF-IDF sum value 
  max_key = max(sum_tf_ids(name), key = sum_tf_ids(name).get) # gives the key of the largest TF-IDF sum value 
  results.append(max_key)
#results
name_label = list(zip(content,results))
name_label

[("11/1 is typically the last standard freeze date across all populations, so it's not unreasonable for schools to have the CDS ready for stakeholder review on that day.",
  'timing'),
 ('I think that the current template formats are past their prime.',
  'template'),
 ('The timing is fine.', 'timing'),
 ('release date is way too late - needs to be released in September or earlier to collect information across departments  explore a common submission format for the publishers such as a standardized XML to eliminate re-typing',
  'timing'),
 ('Excel template has many rows not adjusted properly to display all text.',
  'template'),
 ("I know they often offer the option of having them manually key-in our data into their systems, but I don't know of other institutions that trust that process so we end up spending a lot of time typing in to the US News Best Colleges, Princeton Review and College Board BigFuture portals.",
  'template'),
 ('I find the CDS extremely useful in gathering common

In [40]:
df_test_predict = pd.DataFrame (name_label, columns = ['responses','label_predict'])
df_test_predict


,responses,label_predict
0,11/1 is typically the last standard freeze dat...,timing
1,I think that the current template formats are ...,template
2,The timing is fine.,timing
3,release date is way too late - needs to be rel...,timing
4,Excel template has many rows not adjusted prop...,template
5,I know they often offer the option of having t...,template
6,I find the CDS extremely useful in gathering c...,timing
7,After fall census and prior to spring ranking ...,timing
8,This would allow us to make sure we collect ne...,template
9,The main feedback I have is to request that AD...,template_accessibility


In [41]:
# join two dataframes in q1_test
#df_test = pd.DataFrame(test_array, columns = ['responses'])
inner_join_df= pd.merge(data_test, df_test_predict, on='responses', how='inner')
inner_join_df.set_index('id')


,responses,label_predict
id,,
1_2,11/1 is typically the last standard freeze dat...,timing
1_4,I think that the current template formats are ...,template
8_1,The timing is fine.,timing
11_1,release date is way too late - needs to be rel...,timing
16_4,Excel template has many rows not adjusted prop...,template
16_7,I know they often offer the option of having t...,template
17_1,I find the CDS extremely useful in gathering c...,timing
18_2,After fall census and prior to spring ranking ...,timing
20_3,This would allow us to make sure we collect ne...,template


In [42]:
inner_join_df.to_excel("/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/q1_label_predicted.xlsx") 

# Calculate the sentence embeddings for the train data and the test data:
Create a function called sent_embeddings() to generate the sentence embeddings using the Sentence Transformer, and another function knn_manual_search() to compare the cosine similarity of each test sentence and a train sentence. The output of the knn_manual_search() gives us the predicted label wich has the highest cosine similarity among all the labels. Finally, the predicted labels were merged to the original test data, so that I could correct the lables and calculate the accuracy rate. The label correction and the accuracy rate was presented in a spreadsheet named "q1_embeddings_label_predicted". It was provided along with the script as well.

In [43]:
model = SentenceTransformer('all-mpnet-base-v2')

In [44]:
# create a function to convert sentences into sentence embeddings
def sent_embeddings(file_name):
  filepath = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/"+file_name+".txt"
  f = open(filepath, "r")
  file = f.read().splitlines()
  sentences_embeddings = []
  for sentence in file:
    embeddings = model.encode(sentence)
    sentences_embeddings.append(embeddings)
  return sentences_embeddings
#type(sentences_embeddings) # sentence_embeddings is a list
#type(sentences_embeddings[0]) # In the list, each element is an array(e.g.,sentences_embeddings[0])


In [24]:
# convert test sentences into sentence embeddings.
#sent_embeddings("q1_test")

In [45]:
# convert train sentences into sentence embeddings
file_names = ["timing","template","template_access","new_platform","prep_method","others"]
sent_embeddings_list = [] 
for file in file_names:
  result = sent_embeddings(file) # create a sentence embeddings list for ONE file only.
  sent_embeddings_list.append(result) # [[[embed array_sent1],[embed array_sent2],[]...],[],[]...]
len(sent_embeddings_list)

6

In [46]:
# write a function called knn_manual_search to find the max cosine similarity
def knn_manual_search(sentence): # enter one test sentence at a time to the function.
  embeddings = model.encode(sentence) # convert one sentence into an array of sentence embeddings.
  cosine_scores_list = []
  dictionary = {}
  for i in range(len(sent_embeddings_list)): # i is a list of lists of arrays of sentence embeddings in a train file.
    for j in sent_embeddings_list[i]: # j refers to an array of sentence ebbeddings in file i.
      cosine_scores = util.cos_sim(embeddings, j)[0] # compare an array (test sentence) to an array (train sentence from a file)
      cosine_scores_list.append(cosine_scores) # collect all the cosine scores for file i.
    max_score = max(cosine_scores_list) # for each file i, find the max cosine score.
    if i == 0:
      dictionary["timing"] = max_score
    elif i == 1:
      dictionary["template"] = max_score
    elif i == 2:
      dictionary["template_accessibility"] = max_score
    elif i == 3:
      dictionary["new_platform"] = max_score
    elif i == 4:
      dictionary["CDS_preparation_method"] = max_score
    else:
      dictionary["others"] = max_score
  max_key = max(dictionary, key = dictionary.get) # gives the key of the largest cosine similarity value.
  return max_key


In [48]:
file_name = "q1_test"
filepath = "/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/"+file_name+".txt"
f = open(filepath, "r")
file = f.read().splitlines()
#file
#type(file) # a list
dict_label = {}
for sentence in file:
  label_predict = knn_manual_search(sentence)
  dict_label[sentence] = label_predict
dict_label

{"11/1 is typically the last standard freeze date across all populations, so it's not unreasonable for schools to have the CDS ready for stakeholder review on that day.": 'timing',
 'I think that the current template formats are past their prime.': 'template_accessibility',
 'The timing is fine.': 'timing',
 'release date is way too late - needs to be released in September or earlier to collect information across departments  explore a common submission format for the publishers such as a standardized XML to eliminate re-typing': 'timing',
 'Excel template has many rows not adjusted properly to display all text.': 'template',
 "I know they often offer the option of having them manually key-in our data into their systems, but I don't know of other institutions that trust that process so we end up spending a lot of time typing in to the US News Best Colleges, Princeton Review and College Board BigFuture portals.": 'new_platform',
 'I find the CDS extremely useful in gathering common data

In [49]:
df_test = pd.DataFrame(dict_label.items(), columns=['responses', 'label_predicted'])
df_test

,responses,label_predicted
0,11/1 is typically the last standard freeze dat...,timing
1,I think that the current template formats are ...,template_accessibility
2,The timing is fine.,timing
3,release date is way too late - needs to be rel...,timing
4,Excel template has many rows not adjusted prop...,template
5,I know they often offer the option of having t...,new_platform
6,I find the CDS extremely useful in gathering c...,new_platform
7,After fall census and prior to spring ranking ...,timing
8,This would allow us to make sure we collect ne...,CDS_preparation_method
9,The main feedback I have is to request that AD...,template_accessibility


In [50]:
# join two dataframes in q1_test
#df_test = pd.DataFrame(test_array, columns = ['responses'])
df_embeddings= pd.merge(data_test, df_test, on='responses', how='inner')
df_embeddings.set_index('id')

,responses,label_predicted
id,,
1_2,11/1 is typically the last standard freeze dat...,timing
1_4,I think that the current template formats are ...,template_accessibility
8_1,The timing is fine.,timing
11_1,release date is way too late - needs to be rel...,timing
16_4,Excel template has many rows not adjusted prop...,template
16_7,I know they often offer the option of having t...,new_platform
17_1,I find the CDS extremely useful in gathering c...,new_platform
18_2,After fall census and prior to spring ranking ...,timing
20_3,This would allow us to make sure we collect ne...,CDS_preparation_method


In [51]:
df_embeddings.to_excel("/content/drive/MyDrive/Colab Notebooks/cds_survey_analysis/cds_results/q1_embeddings_label_predicted.xlsx") 

# Discussion:
The accuracy rate using the TF-IDF score for labeling is 0.71, while that using the sentence embeddings for labeliing is 0.77. A possible reason that the sentence embeddings method is better than the TF-IDF method is that the sentence embeddings are based on certain features of the words, while TF-IDF score is based on the occurrences of words. The method based on the occurrences of words has a problem that it works well when the content of the text is general, not domain-specific, but when it deals with domain-specific texts, certain important words of the domain can be mentioned many times across the texts, but because they occur almost everywhere, the imporance of the words will be reduced by the TF-IDF method. 

In this case, the survey is domain-specific, some important words such as CDS, release date, template, etc. occur in many sentences. According to the formula of TF-IDF, the importance of these words can be reduced, affecting the acuracy of the prediction. 

This project collects only 159 sentences. The scale of the dataset is small. To improve the model, more data points can be added in the future when more survey responses are collected. Other ways to improve the model includes:
1. On the stage of data cleaning, the stopwords, punctunations, and non-utf characters can be removed before further analysis.  
2. Use HuggingFace's default part of speech (pos) tagger, a variant of the token classification task, to identify and extract only nouns, proper nouns , and adjectives. These words are usually more meaningful than others.
3. After the predicted labels of some sentences were corrected, analyze the reason that some sentences were predicted incorrectly. If necessary, create new labels for those sentences and run the model again. This step may be repeated a few times until the accuracy rate is satisfied. 